# LaZerRanks Glicko

In [1]:
using JuliaDB, FileIO, IterableTables, CSVFiles, CSV;|

  likely near In[1]:3
  likely near In[1]:6
  likely near In[1]:7
  likely near In[1]:10


| (generic function with 50 methods)

In [2]:
IndexedTables.set_show_compact!(false)

false

In [3]:
function load_elapsed_table()
    path_comps = "WCA_export_Competitions.tsv"
    wca_comps = loadndsparse(path_comps; delim = '\t', quotechar = '\\',
        colparsers=Dict("id"=>String, "name"=>String, "year"=>Int, 
            "month"=>Int, "day"=>Int, "endMonth"=>Int, "endDay"=>Int))
    function comp_date(c)
        Date(c[6], c[7], c[8])
    end
    function days_elapsed(comp)
         Date(now()) - comp_date(comp)
    end
    comp_ids = []
    comp_elapsed = []
    for i in 1:length(wca_comps)
        c = wca_comps[i]
        push!(comp_ids, c[1])
        push!(comp_elapsed, days_elapsed(c).value)
    end
    
    elapsed_table = table(comp_ids, comp_elapsed, names=[:competitionId, :elapsed], pkey = :elapsed)
    return elapsed_table
end

load_elapsed_table (generic function with 1 method)

In [4]:
function load_elapsed_sparse()
    table = load_elapsed_table()
    comp_ids = []
    comp_elapsed = []
    for i in 1:length(table)
        c = table[i]
        push!(comp_ids, c[1])
        push!(comp_elapsed, c[2])
    end
    elapsed_sparse = ndsparse(@NT(competitionId=comp_ids), comp_elapsed)
end    

load_elapsed_sparse (generic function with 1 method)

In [5]:
elapsed_table = load_elapsed_table()
elapsed_sparse = load_elapsed_sparse()

1-d NDSparse with 5566 values (Any):
competitionId                  │
───────────────────────────────┼─────
"100Merito2018"                │ 253
"10doRioGrandedoNorte2018"     │ 8
"12SidesofSilesia2018"         │ 295
"150thCubeMeetinginBrest2017"  │ 519
"1AVG2013"                     │ 2092
"1BodyCubing2017"              │ 568
"1stDibrugarhOpen2018"         │ 43
"2AVG2014"                     │ 1785
"2FTISanDiego2016"             │ 743
"2ndGuwahatiOpen2016"          │ 946
"2ndJogjaMiniCompetition2015"  │ 1120
"2ndNegrosSpeedcubingOpen2017" │ 561
                               ⋮
"ZurichOpen2016"               │ 946
"ZurichOpen2017"               │ 463
"ZurichOpen2018"               │ 225
"ZurichOpen2019"               │ -146
"ZwolleOpen2017"               │ 491
"ZwolleOpen2018"               │ 128
"chilerubik2011"               │ 2899
"iCubeDay2017"                 │ 532
"iCubeOpen2017"                │ 421
"sydchamps2017"                │ 372
"taxco2018"                    │ 29

In [6]:
results = loadndsparse("WCA_export_Results.tsv"; delim = '\t',
          indexcols = [:competitionId, :eventId, :roundTypeId, :personId],
          colparsers=Dict("personId"=>String,"eventId"=>String,"competitionId"=>String, "roundTypeId"=>String, "pos" =>Int))

4-d NDSparse with 1965270 values (13 field named tuples):
competitionId    eventId  roundTypeId  personId     │ pos  best  average  personName                            personCountryId  formatId  value1  value2  value3  value4  value5  regionalSingleRecord  regionalAverageRecord
────────────────────────────────────────────────────┼─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
"100Merito2018"  "222"    "1"          "2007CINO01" │ 6    392   772      "Rafael Werneck de Andrade Cinoto"    "Brazil"         "a"       392     829     679     809     1495    ""                    ""
"100Merito2018"  "222"    "1"          "2014SANT16" │ 2    565   586      "Gabriel Silva dos Santos"            "Brazil"         "a"       565     601     586     668     570     ""                    ""
"100Merito2018"  "222"    "1"          "2015AGUI11" │ 15   1329  -1       "Luís Otávio L

In [7]:
function genGlickoArray(comp, event, round)
    myresults = table(results[comp, event, round, :])
    myresults_table = table(select(myresults, (:personId, :pos)), pkey=:pos)
    output = []
    for i in 1:length(myresults_table), j in (i+1):length(myresults_table)
        o = [myresults_table[i].personId, myresults_table[j].personId, 1]
        o2 = [myresults_table[j].personId, myresults_table[i].personId, 0]
        push!(output, o)
        push!(output, o2)
    end
    #=
    p1s = []
    p2s = []
    scs = []
    for i in 1:length(output)
        push!(p1s, output[i][1])
        push!(p2s, output[i][2])
        push!(scs, output[i][3])
    end    
    output_table = table(p1s, p2s, scs, names=[:p1,:p2,:sc])
    FileIO.save("output_$comp$event$round.csv", output_table)
    =#
    return output
end
    

genGlickoArray (generic function with 1 method)

In [14]:
function genAllGlickoArrays(event)
    comps = []
    for i in 1:length(elapsed_table)
        push!(comps, elapsed_table[i][1])
    end
    comps = reverse(comps)
    allGlickoArrays = []
    lengthOfComps = length(comps)
    for i in 1:length(comps)
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "0"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "1"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "2"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "3"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "b"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "c"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "d"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "e"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "f"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "g"))
        push!(allGlickoArrays, genGlickoArray(comps[i], event, "h"))
        print("\r$i / $lengthOfComps Comps")
    end
    output = []
    for i in 1:length(allGlickoArrays)
        if length(allGlickoArrays[i]) != 0
            push!(output, allGlickoArrays[i])
        end
    end
    return output
end

genAllGlickoArrays (generic function with 1 method)

In [15]:
@time thing = genAllGlickoArrays("333")

5566 / 5566 Comps108.370099 seconds (257.86 M allocations: 13.401 GiB, 24.45% gc time)


14928-element Array{Any,1}:
 Any[Any["1982THAI01", "1982RAZO01", 1], Any["1982RAZO01", "1982THAI01", 0], Any["1982THAI01", "1982LABA01", 1], Any["1982LABA01", "1982THAI01", 0], Any["1982THAI01", "1982PETR01", 1], Any["1982PETR01", "1982THAI01", 0], Any["1982THAI01", "1982UENO01", 1], Any["1982UENO01", "1982THAI01", 0], Any["1982THAI01", "1982JEAN01", 1], Any["1982JEAN01", "1982THAI01", 0]  …  Any["1982GALR01", "1982TENE01", 1], Any["1982TENE01", "1982GALR01", 0], Any["1982GALR01", "1982TRAJ01", 1], Any["1982TRAJ01", "1982GALR01", 0], Any["1982SEBE01", "1982TENE01", 1], Any["1982TENE01", "1982SEBE01", 0], Any["1982SEBE01", "1982TRAJ01", 1], Any["1982TRAJ01", "1982SEBE01", 0], Any["1982TENE01", "1982TRAJ01", 1], Any["1982TRAJ01", "1982TENE01", 0]]
 Any[Any["2003KNIG01", "2003AKIM01", 1], Any["2003AKIM01", "2003KNIG01", 0], Any["2003KNIG01", "2003ALLE01", 1], Any["2003ALLE01", "2003KNIG01", 0], Any["2003KNIG01", "2003VAND01", 1], Any["2003VAND01", "2003KNIG01", 0], Any["2003KNIG01", "1982

In [16]:
path_persons = "WCA_export_Persons.tsv"
    wca_persons = loadtable(path_persons; delim = '\t', quotechar = '\\',
        colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

In [17]:
persons = loadndsparse(path_persons; delim = '\t', quotechar = '\\', indexcols = [:id, :subid],
        colparsers=Dict("id"=>String,"subid"=>Int, "name"=>String, "CountryId"=>String, "gender"=>String));

In [18]:
playerids = []
for i in 1:length(wca_persons)
    push!(playerids, wca_persons[i][1])
end

In [19]:
#reset ratings
ratings = []
RDs = []
vols = []
for i in 1:length(playerids)
    push!(ratings, 1500.0)
    push!(RDs, 350.0)
    push!(vols, 0.06)
end

In [20]:
ratingdict = Dict(zip(playerids, ratings))
RDdict = Dict(zip(playerids, RDs))
voldict = Dict(zip(playerids, vols));

In [21]:
function calcRating(p1,p2,sc)
    tau = 0.5
    g(phi) = (1 + (3*(phi^2)*2)/(2*(pi^2)))^(-1/2)
    E(mu1, mu2, phi2) = 1/(1 + e^(-g(phi2)*(mu1 - mu2)))
    f(x) = (e^x)*((delta^2 - phi1^2 - v - e^x)/(2*(phi1^2 + v + e^x)^2)) - (x-a)/((tau)^2)
    
    r1 = ratingdict[p1]
    r2 = ratingdict[p2]
    RD1 = RDdict[p1]
    RD2 = RDdict[p2]
    sigma1 = voldict[p1]
    
    mu1 = (r1 - 1500)/173.7178
    phi1 = RD1/173.7178
    mu2 = (r2 - 1500)/173.7178
    phi2 = RD2/173.7178
    
    v = ((g(phi2)^2)*E(mu1, mu2, phi2)*(1 - E(mu1, mu2, phi2)))^-1
    delta = v*(g(phi2)*(sc - E(mu1, mu2, phi2)))
    epsilon = 0.000001
    
    A = a = log(sigma1^2)
    if delta^2 > phi1^2 + v
        B = log(delta^2 - phi1^2 - v)
    else
        k = 1
        while f(a - k*tau) < 0
            k += 1
        end
        B = a - k*tau 
    end
    fA = f(A)
    fB = f(B)
    while abs(B-A) > epsilon
        C = A + ((A - B)*fA)/(fB - fA)
        fC = f(C)
        if fC*fB < 0 
            A = B
            fA = fB
        else
            fA = fA/2
        end
        B = C
        fB = fC
    end
    
    sigma1prime = e^(A/2)
    phistar = sqrt(phi1^2 + sigma1prime^2)
    phi1prime = 1/sqrt(1/(phistar^2) + 1/v)
    mu1prime = mu1 + (phi1prime^2)*(g(phi2)*(sc - E(mu1, mu2, phi2)))
   
    r1prime  = 173.7178*mu1prime + 1500
    RD1prime = 173.7178*phi1prime
    
    ratingdict[p1] = r1prime
    RDdict[p1] = RD1prime
    voldict[p1] = sigma1prime
end

calcRating (generic function with 1 method)

In [22]:
function calcRatings(data)
    for i in 1:length(data)
        calcRating(data[i][1], data[i][2], data[i][3])
    end
end    

calcRatings (generic function with 1 method)

In [23]:
@time for i in 1:length(thing)
    calcRatings(thing[i])
    print("\r$i / ", length(thing))
end

14928 / 14928629.033554 seconds (9.70 G allocations: 144.667 GiB, 22.74% gc time)


In [24]:
RatingList = sort(collect(zip(values(ratingdict),keys(ratingdict))), rev = true)

116190-element Array{Tuple{Any,Any},1}:
 (5138.22, "2012PARK03") 
 (4718.81, "2010WEYE01") 
 (4695.83, "2009ZEMD01") 
 (4685.77, "2011ETTE01") 
 (4529.73, "2010WANG68") 
 (4484.05, "2013NAHM01") 
 (4465.15, "2014XUZI01") 
 (4289.13, "2012PONC02") 
 (4175.6, "2012LEWI01")  
 (4175.44, "2010WEYE02") 
 (4171.32, "2011DOBR01") 
 (4117.39, "2015CHEA01") 
 (4102.03, "2012PATE01") 
 ⋮                       
 (-1109.28, "2013KEYV01")
 (-1146.91, "2016ZHAO47")
 (-1205.13, "2016ZHAJ03")
 (-1216.9, "2014GUIL01") 
 (-1229.27, "2016WANH11")
 (-1241.23, "2016ZHEN34")
 (-1253.26, "2016ZHAY04")
 (-1265.36, "2016HAOM01")
 (-1277.52, "2016XUZH07")
 (-1289.75, "2016LIZH06")
 (-1302.04, "2016ZHAO44")
 (-1314.41, "2016LIQI04")

In [32]:
for i in 1:length(RatingList)
    id = RatingList[i][2]
    name = persons[id, 1].name
    rating = RatingList[i][1]
    RD = RDdict[id]
    @printf("\n %d %10s %-30s %-4.2f   %-3.4f", i, id, name, rating, RD)
end


 1 2012PARK03 Max Park                       5138.22   329.3496
 2 2010WEYE01 Philipp Weyer                  4718.81   203.4506
 3 2009ZEMD01 Feliks Zemdegs                 4695.83   192.9460
 4 2011ETTE01 Lucas Etter                    4685.77   90.8022
 5 2010WANG68 Bill Wang                      4529.73   200.2608
 6 2013NAHM01 Seung Hyuk Nahm (남승혁)       4484.05   218.0880
 7 2014XUZI01 Zibo Xu (徐子博)               4465.15   179.1493
 8 2012PONC02 Patrick Ponce                  4289.13   194.2474
 9 2012LEWI01 Phillip Lewicki                4175.60   229.7169
 10 2010WEYE02 Sebastian Weyer                4175.44   145.7764
 11 2011DOBR01 Dmitry Dobrjakov               4171.32   144.3372
 12 2015CHEA01 Andrey Che                     4117.39   125.2573
 13 2012PATE01 Antonie Paterakis              4102.03   140.3491
 14 2007VALK01 Mats Valk                      4075.71   137.6649
 15 2010WANG53 Jiayu Wang (王佳宇)            4053.03   187.3356
 16 2014KIMD06 Doyul Kim (김도율)             

 154 2012CHEN28 Yifei Chen (陈奕霏)            3484.80   131.0928
 155 2010UTOM01 Vincent Hartanto Utomo         3480.01   145.2646
 156 2011AMOR01 David Remolina Amórtegui       3478.26   129.0840
 157 2018LIUZ14 Zhenyang Liu (刘振洋)          3470.09   97.0520
 158 2017LIZI06 Zixi Li (李梓熙)               3470.07   112.9040
 159 2013GERT01 Nicolas Gertner                3467.73   103.7557
 160 2018HUAN08 Hua Huang (黄华)               3466.11   119.7539
 161 2015OAKK01 Kunal Oak                      3465.73   107.4558
 162 2010JINY02 Yan Jin (金岩)                 3465.63   106.4720
 163 2014JOHN04 Alex Johnson                   3455.51   102.5310
 164 2017YANG55 Zixuan Yang (杨子轩)           3454.45   117.9364
 165 2015GUNN01 Jakob Gunnarsson               3448.76   100.7685
 166 2014MARS04 Ty Marshall                    3446.23   104.0654
 167 2014LOKW01 Wong Kin Lok (黃健樂)          3445.18   161.8337
 168 2013KAMI03 Vladislav Kaminskiy            3438.95   147.2173
 169 2013BRAT01 Martin Brath  

 305 2014OMIA01 Igor Ośmiałowski               3241.38   91.4408
 306 2015DEGL01 Lucas Déglise                  3241.35   100.9690
 307 2016CARO06 Marcos Aparicio Caro           3240.72   144.2627
 308 2013BARK01 Justin Barker                  3239.88   128.8962
 309 2016TWYM01 Seamus Twyman                  3239.88   120.5256
 310 2015ORTE02 Nathaniel Ortega               3239.82   82.4930
 311 2010SCHI01 Reinier Schippers              3238.94   131.1729
 312 2011ZENG06 Xiaolin Zeng (曾小林)          3237.77   98.8776
 313 2015DIZO02 Kieran Dizon                   3235.32   102.9694
 314 2011CHEN19 Qingbin Chen (陈庆斌)          3235.32   103.8295
 315 2012SHAK01 Erik Shakirov                  3234.98   161.1912
 316 2012WICH01 Bennet Wichmann                3233.69   146.9559
 317 2016GARC14 Carlos Cesar Rodríguez García  3232.77   99.5693
 318 2013WALL03 Daniel Wallin                  3230.73   100.1360
 319 2017ZHAQ04 Qiyu Zhang (张淇奥)            3230.21   95.6499
 320 2010FUEN01 Leopoldo

 535 2016ZHUW02 Wangkang Zhu (朱旺康)          3046.40   101.9270
 536 2016GIUL01 Alessio Giuliano               3045.32   100.8535
 537 2018HUAN68 Dongli Huang (黄东理)          3044.60   90.4957
 538 2016HAYA02 Taisho Hayashi (林大翔)        3044.41   95.9456
 539 2013KLEI03 Livia Kleiner                  3043.40   83.4304
 540 2011ZALE02 Marcin Zalewski                3042.16   110.4955
 541 2014LANG02 Isaac Langer                   3041.85   89.8528
 542 2016MANN01 Evan Mann                      3041.59   89.0257
 543 2018LIUM02 Mingyu Liu (刘铭宇)            3041.05   104.1876
 544 2013ZENG02 Fangrui Zeng (曾方锐)          3038.61   117.3952
 545 2017FARA02 Amirali Farajnezhad (امیرعلی فرج نژاد) 3037.36   94.0978
 546 2011FATE01 Seyyed Mohammad Hossein Fatemi (سید محمد حسین فاطمی) 3036.12   101.4879
 547 2015YAKH01 Daniel Yakhkind                3034.43   79.4723
 548 2017ROMA11 Edson Bryan Béjar Román        3031.36   131.1783
 549 2015MEND08 Ramiro Mendez                  3030.48   142.4081
 5

 686 2014BHUT01 Aman Bhutani                   2949.26   98.6231
 687 2015DRIS02 Ryan Driscoll                  2948.64   97.7539
 688 2018BERE01 János Bereczki                 2948.40   110.4625
 689 2016XIAO04 Qingyuan Xiao (肖庆源)         2947.52   104.3527
 690 2010WAND03 Alfrisa Diva Wandana           2946.73   110.8060
 691 2013LIUJ03 Jialong Liu (刘嘉龙)           2946.34   75.6657
 692 2016LURU01 Ruikang Lu (陆瑞康)            2945.98   195.2007
 693 2013LEET01 Tsung-Che Lee (李宗哲)         2945.46   79.0060
 694 2015WALK03 Micah Walker                   2944.90   98.0973
 695 2014ENDO01 Jolo Endona                    2944.15   75.7264
 696 2010STOL01 Cameron Stollery               2943.36   80.7439
 697 2011HYEO01 You Hyeon-Dong (유현동)        2943.06   107.3483
 698 2011ERYO01 Alexander Eryomin              2943.01   96.9710
 699 2014TURY01 Danyil Turyk (Даниїл Турик)    2942.04   88.5725
 700 2009SALA01 Jorge Leonardo Sánchez Salazar 2941.63   95.3407
 701 2013SEKH02 Pradyut Sekhsaria  

...

 7255 2016KOKA01 Sarthak Kokane                 1994.91   73.7355
 7256 2015COLL08 Jonathan Collins               1994.78   116.0697
 7257 2015ASRA01 Chirag Asrani                  1994.63   71.4020
 7258 2010ANSO01 Muhammad Isa Ansori            1994.63   74.7036
 7259 2013JEZI01 Szymon Jeziorski               1994.54   92.6354
 7260 2015MILL08 Derick Miller                  1994.50   73.8254
 7261 2018COSA01 Umut Coşan                     1994.46   86.0289
 7262 2009XIAT01 Tian Xia (夏天)                1994.32   75.8792
 7263 2018GUOJ01 Jiahao Guo (郭佳豪)            1994.20   100.8232
 7264 2015QUIN06 Elias Quintos                  1994.18   74.8349
 7265 2014KIMM01 Minseok Kim                    1994.15   83.8942
 7266 2016CHEY13 Yongkang Chen (陈永康)         1994.15   76.4598
 7267 2013CHEN87 Andong Chen (陈安栋)           1994.10   78.0765
 7268 2005DOMO01 Takahito Domon (土門崇人)      1994.06   77.2863
 7269 2018ALMA03 Askar Almaz (Аскар Алмаз)      1994.06   83.0798
 7270 2012FERR06 Davide

 7425 2014BOIK01 Artem Boiko (Артем Бойко)      1983.03   72.1094
 7426 2018LIJI07 Jintao Li (李进滔)             1983.00   75.5305
 7427 2011FLAY02 Matthew Flay                   1982.89   91.4229
 7428 2016SVEC01 Jozef Švec                     1982.84   86.4686
 7429 2016XION02 Yangshuo Xiong (熊阳硕)        1982.81   64.3177
 7430 2014LARI01 Adil Lari                      1982.72   78.9943
 7431 2016AOYA01 Yuto Aoyagi (青柳友都)         1982.35   105.8958
 7432 2017CHZH01 Konstantin Chzhen              1982.22   108.1935
 7433 2017BUBL01 Sergey Bublikov                1982.12   73.5664
 7434 2010TARE01 Teera Tareesuchevakul (ธีรา ธารีสุชีวกุล) 1982.12   74.8349
 7435 2017CALD04 Roberto da Costa Barbosa Nunes Caldas 1982.10   70.5939
 7436 2015KOKK01 Antonis Kokkinakis             1982.01   77.0752
 7437 2018MIKH01 Mikhail Gitlin                 1981.91   80.5354
 7438 2013HUAN29 Gianfranco Huanqui             1981.87   75.1742
 7439 2015SANT06 Rafael Lucas Leão Santos       1981.82   71.5667


 7575 2016RIXS01 Jonathan Rixstine              1973.25   78.0874
 7576 2017TOAP01 David Israel Chiguano Toapanta 1973.22   72.6500
 7577 2017BHAT02 Atharva R. Bhat                1973.18   74.0824
 7578 2015WYSS01 Dimitri Wyss                   1973.13   86.1832
 7579 2015LAIS02 Simon Lai                      1973.13   107.4139
 7580 2015CHEN51 Daniel Chen                    1973.11   76.1667
 7581 2016STAR02 Caleb Starr                    1973.03   79.0335
 7582 2015RAVI02 Aditya Ravindran               1972.96   92.6151
 7583 2017GAMA01 Ismael Benjamin Ñiquin Gamarra 1972.92   89.9465
 7584 2011LIUZ01 Zhenyu Liu (刘振宇)            1972.81   79.5801
 7585 2006MATS03 Yoshihiko Matsushita (松下佳彦) 1972.79   77.0072
 7586 2017ZHAN03 Changfeng Zhang (张昌丰)       1972.77   80.3922
 7587 2010TRON01 Santiago Pinedo Troncoso       1972.70   75.6077
 7588 2016GEHR01 Liam Gehrsitz                  1972.68   76.4944
 7589 2011FLAR01 Manuel Alejandro Flórez        1972.65   73.4095
 7590 2013KODA01 S

Excessive output truncated after 524296 bytes.

70.6310
 7727 2012ZOUF01 Fengjun Zou (邹丰骏)           1963.92   77.4333
 7728 2017CARR13 Narlen Oscar Colqui Carrillo   1963.91   69.2130
 7729 